1. paths can be set from here.
2. all the paths are inside the dragvideo container
3. set kernel : Dragvideo env 

# PTI

## some utils

In [17]:
# ========================================
#            UTILITY FUNCTIONS
# ========================================

import os 

def add_dummy_config(filename,text,ROOT_PATH = '.',mode = "w"):
    file_path = os.path.join(ROOT_PATH, filename)
    with open(file_path, mode) as f:
        f.write(text)
        

def generate_text(**kwargs):
    text = "" 
    for key, value in kwargs.items():
        if type(value) == str:
            value = f"'{value}'"
        text += f"{key}= {value}\n"
    return text

def add_dummy_config_from_dict(filename,dict_,ROOT_PATH = '.',mode = "w"):
    text = generate_text(**dict_)
    add_dummy_config(filename,text,ROOT_PATH = ROOT_PATH,mode = mode)
    
    
# ========================================
def import_module_from_path( path,name='_module'):
    # ex: path = "path/to/module.py"
    import importlib.util
    spec = importlib.util.spec_from_file_location(name, path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module


## set paths here

<b> NOTE:</b>  
To update the configs. this file changes must happen before import.
restart <i>to not import </i> from cached files

In [18]:
# this is must
import os
os.chdir('..')
print(os.getcwd())

/home/bean/DragVideo


input experiment path here...

In [19]:
# create experiment data folder structure
Experiment_name = 'exp_01_man'
Experiment_base_path = '/home/bean/DragVideo/Data_store/experiments/' 
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

import _PTI as pti
pti.init_experiment_dir(Experiment_name,Experiment_base_path)

Experiment directory already exists


In [20]:
# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": "/home/bean/DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
    "stylegan2_ada_ffhq": "/home/bean/DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
}

# dummy paths_config overwrites the paths_config.py
dummy_config_path = '/home/bean/DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=dummy_config_path)

## preprocessing

In [21]:
from utils_draggan import video_utils

# get frames 
video_path = "/home/bean/DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4"
raw_path = os.path.join(Experiment_path, "raw")

# video_utils.extract_frames(video_path, raw_path)
# pti.pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path 

use_multi_id_training = True
model_id = pti.run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

# print(f'{model_id=}')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/DragVideo/env/Dragvideo/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo
current_directory:  /home/bean/DragVideo
torch.Size([3, 1024, 1024]) 3 1024 1024


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 107, in forward
      _48 = torch.__not__(torch.__contains__(_47, "conv1"))
      if _48:
        x5 = (_22).forward(x4, )
              ~~~~~~~~~~~~ <--- HERE
      else:
        x5 = x4
  File "code/__torch__.py", line 447, in forward
    _192 = torch.to(self.bias, ops.prim.dtype(x), False, False, None)
    _193 = self.padding
    x49 = torch.conv2d(x, _191, _192, [1, 1], [_193, _193], [1, 1], 1)
          ~~~~~~~~~~~~ <--- HERE
    x50 = __torch__.torch.nn.functional.relu(x49, False, )
    return x50

Traceback of TorchScript, original code (most recent call last):
  File "c:\p4research\research\tkarras\dnn\gan3support\feature_detectors\vgg.py", line 103, in forward
                    ys.append(x.to(torch.float32))
                if name not in ['pool5', 'fc1', 'fc2', 'fc3', 'softmax']:
                    

change .pt to .pkl file ( .pt has just weights. But .pkl must be complete module)

In [ ]:
import torch 
torch.__version__

'2.0.1+cu117'

In [ ]:
!pip install torch torchvision 

  Using cached torch-2.0.1-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.2-cp39-cp39-manylinux1_x86_64.whl (6.0 MB)


In [ ]:
!nvidia-smi

Thu Aug 10 10:36:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1A:00.0 Off |                  Off |
| 35%   63C    P2    67W / 230W |   6428MiB / 24256MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!conda install pytorch torchvision cudatoolkit=11.1 

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2



## Package Plan ##

  environment location: /home/bean/DragVideo/env/Dragvideo

  added / updated specs:
    - cudatoolkit=11.1
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libprotobuf-3.20.3         |       he621ea3_0         2.4 MB
    pytorch-2.0.1              |cpu_py39hdc00b08_0        65.4 MB
    torchvision-0.15.2         |cpu_py39h83e0c9b_0         9.8 MB
    typing-extensions-4.7.1    |   py39h06a4308_0           9 KB
    ------------------------------------------------------------
              

In [ ]:
# model_id ='QBUXQCXZGWET'

In [ ]:
use_multi_id_training = True

generator_type = pti.paths_config.multi_id_model_type if use_multi_id_training else image_name
old_G, new_G = pti.load_generators(model_id, generator_type)

pti.export_updated_pickle(new_G,model_id,name = 'stylegan2')

model_id # 'QBUXQCXZGWET'

Exporting large updated pickle based off new generator and ffhq.pkl


'QBUXQCXZGWET'

# landmarks

In [ ]:
#get landmarks
landmark_path = "/home/bean/DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)

landmark_path:  /home/bean/DragVideo/Data_store/experiments/exp_01_man/landmarks/000.pkl


# Run DragGAN

In [ ]:
# running draggan

# visualizer_auto.py
# - cuda device setting
# - pretrained_stylegan_weights path -> gloabl_state()

# viz/renderer.py
# - landmark points resolution error fixes

# visualizer_experiment.ipynb
import os
DragGAN_dir = "/home/bean/DragVideo/DragGAN"
os.chdir(DragGAN_dir)

from auto_drag import do_drag
from auto_drag import modify_landmarks

def get_arguments(name,
                  N_STEPS=100,
                  CHECKPOINT_PATH=None): # exp_name
                  #basedir+exp_name
    landmarks_path =os.path.join(landmarks_dir,str(name)+".pkl")# f"/home/bean/DragVideo/Data_store/data/PTI_results/landmarks/{name}.pkl"
    return {
        'w_load_path': os.path.join(Experiment_path,'latents','barcelona','PTI') + "/{name}/0.pt",
        'stylegan2_wieghts_path' : CHECKPOINT_PATH,

        'points' : modify_landmarks(landmarks_path),
        'N_STEPS': N_STEPS,
        'save_path': os.path.join(Experiment_path,'after_drag')+"/{name}.png",
    }

In [ ]:
import os
import pickle

latents_dir = os.path.join(Experiment_path,'latents') + "/barcelona/PTI"

#  run draggan on based on latents availability

temp = os.listdir(latents_dir)
temp.sort()
names = [i.split('.')[0] for i in temp]
# names

# path to the desired stylegan finetuned weights from PTI
TUNED_STYLEGAN3_WEIGHTS_PATH =  os.path.join(Experiment_path,'tuned_SG') +"/stylegan3_QBUXQCXZGWET.pkl"

for name in names:
    args = get_arguments(name,CHECKPOINT_PATH =TUNED_STYLEGAN3_WEIGHTS_PATH,N_STEPS=1)
    do_drag(**args)

FileNotFoundError: [Errno 2] No such file or directory: '/home/bean/DragVideo/Data_store/experiments/exp_01_man/latents/barcelona/PTI/{name}/0.pt'

In [ ]:
#print list of imported modules
print("Imported modules:")
import sys
print('\n'.join(sys.modules.keys()))
# create a namespace and put auto_drag module in it.
import auto_drag


Imported modules:
sys
builtins
_frozen_importlib
_imp
_thread
_warnings
_weakref
_io
marshal
posix
_frozen_importlib_external
time
zipimport
_codecs
codecs
encodings.aliases
encodings
encodings.utf_8
_signal
encodings.latin_1
_abc
abc
io
__main__
_stat
stat
_collections_abc
genericpath
posixpath
os.path
os
_sitebuiltins
_locale
_bootlocale
_distutils_hack
types
importlib._bootstrap
importlib._bootstrap_external
warnings
importlib
importlib.machinery
_heapq
heapq
itertools
keyword
_operator
operator
reprlib
_collections
collections
collections.abc
_functools
functools
contextlib
enum
_sre
sre_constants
sre_parse
sre_compile
copyreg
re
typing.io
typing.re
typing
importlib.abc
importlib.util
mpl_toolkits
site
_weakrefset
weakref
pkgutil
runpy
ipykernel._version
_json
json.scanner
json.decoder
json.encoder
json
errno
signal
threading
pwd
grp
_posixsubprocess
select
math
selectors
subprocess
jupyter_client._version
platform
_ctypes
_struct
struct
ctypes._endian
ctypes
zmq.backend.select
_cy

In [ ]:
import os
os.chdir('..')
print(os.getcwd())
import _PTI as p

/home/bean/DragVideo/DragGAN
_path='/home/bean/DragVideo/DragGAN/_PTI/__init__.py'
path='/home/bean/DragVideo/DragGAN/_PTI'


In [ ]:
os.getcwd()

'/home/bean/DragVideo/DragGAN/_PTI'